### Mahesh Kadam Assignment

In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
%matplotlib inline
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import SVC
#from nltk.stem.porter import PorterStemmer
import re

In [5]:
df= pd.read_csv('US-Afghan_war_tweets.csv')
df.head()

,user_location,date,text,sentiment,polarity,hashtags,source
0,NaN,8/19/2021 23:59,cant recall told troops afghanistan falsely ...,negative,-0.300,['afghanistan'],twitter for iphone
1,"washington, dc",8/19/2021 23:59,shocker afghanistan taliban carrying doorto do...,neutral,0.000,['afghanistan'],twitter for ipad
2,NaN,8/19/2021 23:59,drop bass bombs big homie afghanistan talibans,negative,-0.075,"['afghanistan', 'talibans']",twitter for iphone
3,"midnapore, west bengal, uk outreach",8/19/2021 23:59,burqa prices risen markets afghanistans provin...,negative,-0.250,['afghanistan'],twitter web app
4,"york, england",8/19/2021 23:59,uk carved afghanistan s heroin minorities mili...,neutral,0.000,['afghanistan'],twitter for ipad


In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mahesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
dffin = df.drop('polarity',axis=1)
dffin.head()
dffin.shape

(362566, 6)

In [9]:
dffin.describe(include='all')

,user_location,date,text,sentiment,hashtags,source
count,261572,362565,362565,362566,276205,362562
unique,35072,12354,359903,3,74712,415
top,india,8/16/2021 20:22,monday morning total chaos kabul airport gun...,neutral,['afghanistan'],twitter for android
freq,11355,288,468,168244,56964,143342


### 3 unique values for target variable shown count for the same below

In [11]:
dffin['sentiment'].value_counts()

neutral     168244
negative    117863
positive     76459
Name: sentiment, dtype: int64

### only 1 observation contains missing value for text data on which we are predicting our target 

In [12]:
dffin['text'].isnull().sum()

1

In [13]:
dffin = dffin.dropna(subset=['text'])
dffin.shape

(362565, 6)

In [14]:
dffin.isnull().mean()

user_location    0.278551
date             0.000000
text             0.000000
sentiment        0.000000
hashtags         0.238192
source           0.000008
dtype: float64

### we want to preprocess and predict sentiment based on text variable so will preprocess and use for model building

In [15]:
dffin['text'] = dffin['text'].astype(str)

C:\Users\mahesh\AppData\Local\Temp/ipykernel_13704/4034992228.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffin['text'] = dffin['text'].astype(str)


### consider sample data for traning due to machine computational capacity

In [16]:
dffortrain = dffin[0:50000]

### Preprocessing text

### Takes in a string of text, then performs the following:
#### 1. Remove all punctuation
#### 2. Remove all stopwords
#### 3. Return the cleaned text as a list of words

In [35]:
def text_process(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    #words = [PorterStemmer().stem(w) for w in words]
    return words

In [36]:
count = 0
new_processed_text =[]
tweets = list(dffortrain['text'])     
for k in tweets:
    count = count +1
    new_processed_text.append(text_process(k))

dffortrain['processed_text'] = new_processed_text

C:\Users\mahesh\AppData\Local\Temp/ipykernel_13704/3329978925.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffortrain['processed_text'] = new_processed_text


In [37]:
dffortrain

,user_location,date,text,sentiment,hashtags,source,processed_text,processed_text_2
0,NaN,8/19/2021 23:59,cant recall told troops afghanistan falsely ...,negative,['afghanistan'],twitter for iphone,"[cant, recall, told, troops, afghanistan, fals...",cant recal told troop afghanistan fals claim o...
1,"washington, dc",8/19/2021 23:59,shocker afghanistan taliban carrying doorto do...,neutral,['afghanistan'],twitter for ipad,"[shocker, afghanistan, taliban, carrying, door...",shocker afghanistan taliban carri doorto door ...
2,NaN,8/19/2021 23:59,drop bass bombs big homie afghanistan talibans,negative,"['afghanistan', 'talibans']",twitter for iphone,"[drop, bass, bombs, big, homie, afghanistan, t...",drop bass bomb big homi afghanistan taliban
3,"midnapore, west bengal, uk outreach",8/19/2021 23:59,burqa prices risen markets afghanistans provin...,negative,['afghanistan'],twitter web app,"[burqa, prices, risen, markets, afghanistans, ...",burqa price risen market afghanistan provinc a...
4,"york, england",8/19/2021 23:59,uk carved afghanistan s heroin minorities mili...,neutral,['afghanistan'],twitter for ipad,"[uk, carved, afghanistan, heroin, minorities, ...",uk carv afghanistan heroin minor milit guess c...
...,...,...,...,...,...,...,...,...
49996,everywhere,8/17/2021 19:19,biden administration worst us history,negative,"['biden', 'us']",twitter for iphone,"[biden, administration, worst, us, history]",biden administr worst us histori
49997,NaN,8/17/2021 19:19,lives civilians protected us eu started shed c...,neutral,"['us', 'eu', 'ethiopia']",twitter for android,"[lives, civilians, protected, us, eu, started,...",live civilian protect us eu start shed crocodi...
49998,NaN,8/17/2021 19:18,us messed afghanistan want mess beautiful amp ...,positive,"['us', 'afghanistan', 'ethiopia']",twitter for android,"[us, messed, afghanistan, want, mess, beautifu...",us mess afghanistan want mess beauti amp ancie...
49999,ã â¤â¦ã â¤â¿ã â¤â²ã â¥âã â¤â²ã â¥â ã â¤âµã â...,8/17/2021 19:18,us military bosses better job keeping morale h...,positive,"['us', 'military']",twitter for android,"[us, military, bosses, better, job, keeping, m...",us militari boss better job keep moral high am...


### unlist the processed text for spliting in train test dataset

In [38]:
def unlist(list):
    words=''
    for item in list:
        words+=item+' '
    return words

In [39]:
dffortrain['processed_text_2'] = dffortrain['processed_text'].apply(lambda x: unlist(x))

C:\Users\mahesh\AppData\Local\Temp/ipykernel_13704/745024313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dffortrain['processed_text_2'] = dffortrain['processed_text'].apply(lambda x: unlist(x))


In [40]:
#dffortrain.to_csv("clntext.csv")   

In [41]:
X_train, X_test,y_train,y_test = train_test_split(dffortrain['processed_text_2'],dffortrain['sentiment'],test_size=0.2,random_state=40)

### Term Frequency Inverse Document Frequency importance weightage to words reprensenting in vector form for maodel building

In [42]:
tfid = TfidfVectorizer(max_features=5000)
features= tfid.fit_transform(X_train)
features_test = tfid.transform(X_test)

In [43]:
features.shape

(40000, 5000)

### implemented 3 different models

## Naive Bayes

In [44]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(features,y_train)
label_pred= model.predict(features_test)

In [45]:
import sklearn.metrics as m

In [46]:
m.accuracy_score(y_test,label_pred)

0.8279

## SVC

In [47]:
model1 =SVC(kernel='sigmoid', gamma=1,C=1)
model1.fit(features,y_train)
label_pred1= model1.predict(features_test)
m.accuracy_score(y_test,label_pred1)

0.8906

## Logistic Regression

In [48]:
from sklearn.linear_model import LogisticRegression
model2 =LogisticRegression()
model2.fit(features,y_train)
label_pred2= model2.predict(features_test)
m.accuracy_score(y_test,label_pred2)

C:\Users\mahesh\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8805

### from above 3 models SVC gives better Score 89.06%

### using RandomizedSearchCV for tuning hyperparameter(C, gamma, kernel)

In [55]:
#Cs = [0.001, 0.01, 0.1, 1, 10]
#gammas = [0.001, 0.01, 0.1, 1, 10]
param_grid = {"gamma": [0.001, 0.01, 0.1, 1, 10], "C": [0.001, 0.01, 0.1, 1, 10],"kernel": ['rbf', 'poly', 'sigmoid']}

In [56]:
from sklearn.model_selection import StratifiedShuffleSplit
cv = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=10)

In [57]:
from sklearn.model_selection import RandomizedSearchCV
rand_grid = RandomizedSearchCV(model11, param_distributions=param_grid,cv=cv,n_jobs=-1)
rand_grid.fit(features,y_train_smote)

RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=3, random_state=10, test_size=0.2,
            train_size=None),
                   estimator=SVC(C=1, gamma=1), n_jobs=-1,
                   param_distributions={'C': [0.001, 0.01, 0.1, 1, 10],
                                        'gamma': [0.001, 0.01, 0.1, 1, 10],
                                        'kernel': ['rbf', 'poly', 'sigmoid']})

In [58]:
rand_grid.best_params_

{'kernel': 'rbf', 'gamma': 0.1, 'C': 10}

### this step taking too much time for my machine so didt tried different iterations

### To handel Imbalanced data used SMOTE from oversampling

In [49]:
from collections impo
rt Counter
print(Counter(y_train))

Counter({'neutral': 20722, 'positive': 10354, 'negative': 8924})


In [50]:
from imblearn.over_sampling import SMOTE
osamp = SMOTE('auto',n_jobs=-1)
features,y_train_smote = osamp.fit_resample(features,y_train)

C:\Users\mahesh\anaconda3\lib\site-packages\imblearn\utils\_validation.py:587: FutureWarning: Pass sampling_strategy=auto as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


In [51]:
print(Counter(y_train_smote))

Counter({'neutral': 20722, 'negative': 20722, 'positive': 20722})


## Model after treating with imbalance data and hypertunning parameters
## 'kernel': 'rbf', 'gamma': 0.1, 'C': 10

In [59]:
model11 =SVC(kernel='rbf', gamma=0.1,C=10)
model11.fit(features,y_train_smote)
label_pred11= model11.predict(features_test)
m.accuracy_score(y_test,label_pred11)

0.9031

### final tunned SVC model11 for prediction on test dataset 90.31%

In [60]:
print(label_pred11)

['neutral' 'neutral' 'neutral' ... 'positive' 'positive' 'neutral']


In [61]:
print(m.classification_report(y_test,label_pred11))
print(m.confusion_matrix(y_test,label_pred11))

              precision    recall  f1-score   support

    negative       0.91      0.72      0.80      2219
     neutral       0.90      0.98      0.94      5200
    positive       0.92      0.90      0.91      2581

    accuracy                           0.90     10000
   macro avg       0.91      0.87      0.88     10000
weighted avg       0.90      0.90      0.90     10000

[[1599  428  192]
 [  69 5108   23]
 [  97  160 2324]]
